# Google Cloud Platform Project Creation Workbook 
 
Use this workbook to create a google cloud project with everything needed to collect new data and host your own web app. 
 
Prerequisites:  
+ Create Google user account  
+ Create your own personal Google Cloud Project and Enable Billing
    - Enable Free Tier account by seleting "Try it Free" here: [Try Google Cloud Platform for free](https://cloud.google.com/cloud-console)
    - Follow steps to activate billing found here: [Create New Billing Account](https://cloud.google.com/billing/docs/how-to/manage-billing-account#create_a_new_billing_account)
        - Billing account is required for APIs used in this project
        - You will not exceed the $300 free trial setting up this project but make sure to delete the project if you do not want to be charged
        - Take note of project name created because this billing account will be used with the new project
+ Install and initialize Google Cloud SDK by following instructions found here: [Cloud SDK Quickstart](https://cloud.google.com/sdk/docs/quickstart)
+ Set default region and zone following instructions here:

## Step 1 - Check Prequisites Successfully Completed
Check that you have successfully installed and enabled Cloud SDK by running the config list command. If you get an error please refer to Troubleshooting steps found here [Cloud SDK Quickstart](https://cloud.google.com/sdk/docs/quickstart).  
You should see an output that includes your account along with any other configuration setup when using gcloud init

In [162]:
!gcloud config list

[accessibility]
screen_reader = False
[compute]
region = us-central1
zone = us-central1-c
[core]
account = cwilbar04@gmail.com
disable_usage_reporting = True
project = nba-predictions-dev



Your active configuration is: [default]


Update all gcloud components to latest release.

In [163]:
!gcloud components update

Beginning update. This process may take several minutes.

All components are up to date.


## Step 2 - Create GCP Project

###### TO DO: Enter name for new project and biling project then change to Code block and run
###### Note: Proect name must be unique across GCP. If you get error when creating project please change the project name here and try again.
new_project_id = 'YOUR_NEW_UNIQUE_PROJECT_NAME'

In [2]:
new_project_id = 'nba-predictions-prod'

In [167]:
!gcloud projects create {new_project_id}

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/nba-predictions-prod].
Waiting for [operations/cp.7706508669557220987] to finish...
..done.
Enabling service [cloudapis.googleapis.com] on project [nba-predictions-prod]...
Operation "operations/acf.p2-130738074716-3c471071-9f24-4528-8e8e-67d53a1207aa" finished successfully.


**TO DO: Navigate to [Cloud Console](https://console.cloud.google.com/), Change to new project, and enable billing following instructions found here: [Enable Billing](https://cloud.google.com/billing/docs/how-to/modify-project#enable_billing_for_a_project)**

## Step 3 - Enable Necessary Cloud Services

This project uses:
+ BigQuery to Store Model Data 
+ Google Cloud Functions scheduled using Google Cloud Scheduler to Load new Data Daily
+ Google App Engine to Host Website
+ Google Firestore in Native Mode to store data used by the Web Page  
  
List below contains all services needed at time of creation of this workbook. Please add/remove from this list if the names/necessary services have changed.

In [171]:
enable_services_list = [
    'appengine.googleapis.com',
    'bigquery.googleapis.com',
    'bigquerystorage.googleapis.com',
    'cloudapis.googleapis.com',
    'cloudbuild.googleapis.com',
    'clouddebugger.googleapis.com',
    'cloudfunctions.googleapis.com',
    'cloudresourcemanager.googleapis.com',
    'cloudscheduler.googleapis.com',
    'cloudtrace.googleapis.com',
    'compute.googleapis.com',
    'datastudio.googleapis.com',
    'deploymentmanager.googleapis.com',
    'firebaserules.googleapis.com',
    'firestore.googleapis.com',
    'logging.googleapis.com',
    'monitoring.googleapis.com',
    'oslogin.googleapis.com',
    'servicemanagement.googleapis.com',
    'serviceusage.googleapis.com',
    'sql-component.googleapis.com',
    'storage-api.googleapis.com',
    'storage-component.googleapis.com',
    'storage.googleapis.com'    
]

In [172]:
## Services can only be enabled 20 at a time at the time of workbook creation. Use this loop to enable 20 at a time.
for x in range(0,len(enable_services),20):
    !gcloud services enable {' '.join(enable_services[x:(x+20)])} --project={new_project_id}   

Operation "operations/acf.p2-130738074716-2d57f6c4-b755-4e1f-b14d-392c602ef21f" finished successfully.
Operation "operations/acf.p2-130738074716-7812275a-cb95-4bcc-acd4-a232feeaea7e" finished successfully.


In [173]:
!gcloud services list --project={new_project_id}

NAME                                 TITLE
appengine.googleapis.com             App Engine Admin API
bigquery.googleapis.com              BigQuery API
bigquerystorage.googleapis.com       BigQuery Storage API
cloudapis.googleapis.com             Google Cloud APIs
cloudbuild.googleapis.com            Cloud Build API
clouddebugger.googleapis.com         Cloud Debugger API
cloudfunctions.googleapis.com        Cloud Functions API
cloudresourcemanager.googleapis.com  Cloud Resource Manager API
cloudscheduler.googleapis.com        Cloud Scheduler API
cloudtrace.googleapis.com            Cloud Trace API
compute.googleapis.com               Compute Engine API
containerregistry.googleapis.com     Container Registry API
datastore.googleapis.com             Cloud Datastore API
datastudio.googleapis.com            Data Studio API
deploymentmanager.googleapis.com     Cloud Deployment Manager V2 API
firebaserules.googleapis.com         Firebase Rules API
firestore.googleapis.com             Cloud Fi

## Step 4 - Create Necessary Service Accounts

There are four primary service accounts used in this project:  
- **App Engine default service account**
    - This gets created automatically when the App engine API is enabled
    - Generally your_project_id@appspot.gserviceaccount.com  
      
- **Compute Engine default service account**
    - This gets created automatically when the Compute engine API is enabled
    - Generally your_project_number-compute@developer.gserviceaccount.com  
      
- **Cloud Function service account**
    - We create this and add necessary roles below using the Cloud SDK
    - cloudfunction-service-account@your_project_name.iam.gserviceaccount.com
    - This account is used as the service account to run all Cloud Functions in this project  
      
- **CircleCI Service Account**
    - We create this and add necessary roles below using the Cloud SDK
    - circleci-deployer@your_project_name.iam.gserviceaccount.com
    - This account is used in CircleCI for CI\CD to deploy and test App Engine and Cloud Functions

Check what service ccounts are already created (should be the two default ones described above)

In [174]:
!gcloud iam service-accounts list --project={new_project_id}

DISPLAY NAME                            EMAIL                                               DISABLED
App Engine default service account      nba-predictions-prod@appspot.gserviceaccount.com    False
Compute Engine default service account  130738074716-compute@developer.gserviceaccount.com  False


In [175]:
!gcloud iam service-accounts create cloudfunction-service-account \
    --display-name="Cloud Function Service Account" \
    --description="Account used to run all Cloud Functions with necessary BigQuery and Firestore Permissions" \
    --project={new_project_id}

Created service account [cloudfunction-service-account].


In [177]:
!gcloud iam service-accounts create circleci-deployer \
    --display-name="Circle CI Service Account" \
    --description="Account used by Circle CI with necessary permissions to Deploy to Cloud Functions and App Engine" \
    --project={new_project_id}

Created service account [circleci-deployer].


Check service accounts were created successfully and display e-mail needed in the next step

In [178]:
!gcloud iam service-accounts list --project={new_project_id}

DISPLAY NAME                            EMAIL                                                                       DISABLED
App Engine default service account      nba-predictions-prod@appspot.gserviceaccount.com                            False
Compute Engine default service account  130738074716-compute@developer.gserviceaccount.com                          False
Circle CI Service Account               circleci-deployer@nba-predictions-prod.iam.gserviceaccount.com              False
Cloud Function Service Account          cloudfunction-service-account@nba-predictions-prod.iam.gserviceaccount.com  False


Programatically update the roles for the new service accounts using the guide found here: [Programatic Change Access](https://cloud.google.com/iam/docs/granting-changing-revoking-access#programmatic)

In [179]:
# Save policy file in directory above where the repo is saved so that it is not stored to github
file_directory = '..\..\policy.json'

In [180]:
# Write current policy to file directory
!gcloud projects get-iam-policy {new_project_id} --format json > {file_directory}

**If running jupyter notebook run below cell to load and modify policy file.**

In [181]:
import json

with open('..\..\policy.json') as f:
    policy = json.load(f)

def modify_policy_add_role(policy, role, member):
    """Adds a new role binding to a policy."""

    binding = {"members": [member],"role": role }
    policy["bindings"].append(binding)
    return policy

members = [f'serviceAccount:cloudfunction-service-account@{new_project_id}.iam.gserviceaccount.com', 
           f'serviceAccount:circleci-deployer@{new_project_id}.iam.gserviceaccount.com']
roles = {members[0]:['roles/bigquery.user','roles/datastore.user','roles/run.serviceAgent'],
        members[1]:['roles/appengine.deployer','roles/appengine.serviceAdmin','roles/cloudbuild.builds.builder',
                   'roles/cloudfunctions.admin','roles/compute.storageAdmin','roles/iam.serviceAccountUser']}

for member in members:
    for role in roles[member]:
        policy = modify_policy_add_role(policy, role, member)

with open('..\..\policy.json', 'w') as json_file:
    json.dump(policy, json_file)

**If running code direct in console, navigate to file path and add the members and roles below in to the file path**  
**Change "your_project_id" to the name of your project id**

{"members": ["serviceAccount:cloudfunction-service-account@your_project_id.iam.gserviceaccount.com"], "role": "roles/bigquery.user"},  
{"members": ["serviceAccount:cloudfunction-service-account@your_project_id.iam.gserviceaccount.com"], "role": "roles/datastore.user"},  
{"members": ["serviceAccount:cloudfunction-service-account@your_project_id.iam.gserviceaccount.com"], "role": "roles/run.serviceAgent"},  
{"members": ["serviceAccount:circleci-deployer@your_project_id.iam.gserviceaccount.com"], "role": "roles/appengine.deployer"},   
{"members": ["serviceAccount:circleci-deployer@your_project_id.iam.gserviceaccount.com"], "role": "roles/appengine.serviceAdmin"},   
{"members": ["serviceAccount:circleci-deployer@your_project_id.iam.gserviceaccount.com"], "role": "roles/cloudbuild.builds.builder"},   
{"members": ["serviceAccount:circleci-deployer@your_project_id.iam.gserviceaccount.com"], "role": "roles/cloudfunctions.admin"},  
{"members": ["serviceAccount:circleci-deployer@your_project_id.iam.gserviceaccount.com"], "role": "roles/compute.storageAdmin"},  
{"members": ["serviceAccount:circleci-deployer@your_project_id.iam.gserviceaccount.com"], "role": "roles/iam.serviceAccountUser"}

In [182]:
!gcloud projects set-iam-policy {new_project_id} {file_directory}

bindings:
- members:
  - serviceAccount:circleci-deployer@nba-predictions-prod.iam.gserviceaccount.com
  role: roles/appengine.deployer
- members:
  - serviceAccount:circleci-deployer@nba-predictions-prod.iam.gserviceaccount.com
  role: roles/appengine.serviceAdmin
- members:
  - serviceAccount:cloudfunction-service-account@nba-predictions-prod.iam.gserviceaccount.com
  role: roles/bigquery.user
- members:
  - serviceAccount:130738074716@cloudbuild.gserviceaccount.com
  - serviceAccount:circleci-deployer@nba-predictions-prod.iam.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-130738074716@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:circleci-deployer@nba-predictions-prod.iam.gserviceaccount.com
  role: roles/cloudfunctions.admin
- members:
  - serviceAccount:service-130738074716@gcf-admin-robot.iam.gserviceaccount.com
  role: roles/cloudfunctions.serviceAgent
- members:
  

Updated IAM policy for project [nba-predictions-prod].


In [183]:
# Remove policy file 
!del {file_directory}

## Step 5 - Create App Engine Application

In order to deploy a specific application you first need to create a placeholder application. 

**Change YOUR_REGION to your default region**  
See [Regions and Zone](https://cloud.google.com/compute/docs/regions-zones) for more info

In [184]:
!gcloud app create --region=YOUR_REGION --project={new_project_id}

You are creating an app for project [nba-predictions-prod].
cannot be changed. More information about regions is at
<https://cloud.google.com/appengine/docs/locations>.

Creating App Engine application in project [nba-predictions-prod] and region [us-central]....
.................................done.
Success! The app is now created. Please use `gcloud app deploy` to deploy your first app.


## Step 6 - Create BigQuery Dataset

Your new project will need a dataset to store the data if you plan on copying/creating your own repository of data.  

This has to be a unique name per project.  

In my workflows I have named the dataset 'nba' but feel free to change it. Note that if you do change it, then you will also need to change the dataset name in any of the other python scripts in this project appropriately. 

In [6]:
new_project_id = 'nba-predictions-test'

In [7]:
dataset_name = 'nba'
!bq --location=US mk -d \
--description "Stores all National Basketball Association Data. Created using Project Creation workbook found at https://github.com/cwilbar04/nba-predictions/tree/main/notebooks" \
{new_project_id}:{dataset_name}  

Dataset 'nba-predictions-test:nba' successfully created.


## Step 7 - Load Raw BigQuery Tables

There are two options for loading the data to BigQuery:  
1. Navigate to [Initial Load Workbook](https://github.com/cwilbar04/nba-predictions/blob/main/notebooks/NBA%20Data%20Initial%20Load.ipynb) and change start date to desired starting date. For my model I loaded data starting from '10-1-1999'. Always choose a start date in between seasons if you don't want to get partial season data.  
  
2. Copy data from my public data set by running the code blocks below.

